In [1]:
import numpy as np

from tqdm.auto import tqdm

In [2]:
# pip install git+https://github.com/dioscuri-tda/pyEulerCurves.git@multi
import pyEulerCurves as pyecc

if pyecc.__version__ != "0.4.5":
    raise ValueError("wrong version!!")

In [3]:
from distance_utils import *

In [4]:
def Hopf_bifurcation(x, y, B):

    Hopf_bifurcation = np.zeros((x.shape[0], y.shape[0], 2))
    Hopf_bifurcation[:, :, 0] = B * x - y - x * (x**2 + y**2)
    Hopf_bifurcation[:, :, 1] = x + B * y - y * (x**2 + y**2)

    return Hopf_bifurcation


x_min, x_max = -2, 2
y_min, y_max = -2, 2
x_points, y_points = 201, 201

x = np.linspace(x_min, x_max, x_points)
y = np.linspace(y_min, y_max, y_points)
X, Y = np.meshgrid(x, y)

all_examples_Hb = []
b_values = np.array([-1, -0.1, 0.1, 1])
names_Hb = []

# set the seed for reproducibility
rng = np.random.default_rng(seed=42)

for b in b_values:
    # without noisy
    Hb_clean = Hopf_bifurcation(X, Y, b)
    all_examples_Hb.append(Hb_clean)
    names_Hb.append(f"Hopf bifurcation b={b:.1f} (clean)")

    # with noisy (1/3)
    noise_x = rng.normal(loc=0.0, scale=np.abs(X) / 3, size=X.shape)
    noise_y = rng.normal(loc=0.0, scale=np.abs(Y) / 3, size=Y.shape)
    Hb_noisy = Hb_clean.copy()
    Hb_noisy[:, :, 0] += noise_x
    Hb_noisy[:, :, 1] += noise_y

    all_examples_Hb.append(Hb_noisy)
    names_Hb.append(f"Hopf bifurcation b={b:.1f} (noisy)")

In [5]:
len(all_examples_Hb)

8

In [6]:
all_examples_Hb[0].shape

(201, 201, 2)

In [7]:
# compute contributions (just the first 2)
ecp_matrix_Hb = []
trans = pyecc.ECC_from_bitmap(multifiltration=True, workers=1)

for i, ex in tqdm(enumerate(all_examples_Hb[:2])):
    ecc = trans.fit_transform(ex)
    ecp_matrix_Hb.append(trans.contributions_list)

0it [00:00, ?it/s]

(41004, 2)
Parallel part done
Elapsed time: 1.4 seconds
Merged 203 dicts
Elapsed time: 0.1 seconds
(41004, 2)
Parallel part done
Elapsed time: 1.4 seconds
Merged 203 dicts
Elapsed time: 0.1 seconds


In [8]:
# Save to file
for i, ecp in enumerate(ecp_matrix_Hb):
    with open("hopf/hopf_{}".format(i), "w") as f:
        for (x, y), val in ecp:
            f.write(f"{x} {y} {val}\n")

In [9]:
## we need to figure out what are the max and min filtration values, so we can correctly compute the distances

print(np.min([c.min(axis=(0, 1)) for c in all_examples_Hb], axis=0))
print(np.max([c.max(axis=(0, 1)) for c in all_examples_Hb], axis=0))

[-20.75947063 -20.94553705]
[20.75912124 20.50516388]


In [10]:
## lets do from -22 to +22
## numba wants them to be floats
dims = (-22.0, 22.0, -22.0, 22.0)

In [11]:
for c in ecp_matrix_Hb:
    print(len(c))

48891
61765


In [12]:
%%time
d1 = difference_ECP(ecp_matrix_Hb[0], ecp_matrix_Hb[1], dims=dims)
print(d1)

creating ECP matrix of size (43835, 43812)
CPU times: user 33min 58s, sys: 18.9 s, total: 34min 17s
Wall time: 34min 28s


KeyboardInterrupt: 

In [13]:
%%time
d2 = difference_ECP_numba(ecp_matrix_Hb[0], ecp_matrix_Hb[1], dims=dims)
print(d2)

creating ECP matrix of size (43835, 43812)
52215.019948258036
CPU times: user 5.39 s, sys: 7.77 s, total: 13.2 s
Wall time: 18.3 s


In [ ]:
# check the two functions return the same value
assert d1 == d2

## C++ code

you can run terminal commands in a cell using `!`

In [15]:
# compile the code, modify ecp_2d.cpp to change the limits and grid size
! g++ -std=c++11 -O2 -o ecp_dist ecp_2d.cpp

In [16]:
# run
! time ./ecp_dist data/hopf/hopf_0 data/hopf/hopf_1

Combined bounding box: xmin = -22, xmax = 22, ymin = -22, ymax = 22, x_step = 10000, y_step = 10000
L1 distance = 52220.12
./ecp_dist data/hopf/hopf_0 data/hopf/hopf_1  1.40s user 0.24s system 82% cpu 2.003 total


## Distance matrix

In [ ]:
# ecp_difference_matrix_Hb = np.zeros((len(ecp_matrix_Hb), len(ecp_matrix_Hb)))

# # compute difference
# for i in tqdm(range(len(ecp_matrix_Hb))):
#     for j in tqdm(range(i + 1, len(ecp_matrix_Hb))):
#         ecp_difference_matrix_Hb[i, j] = difference_ECP_fast(
#             ecp_matrix_Hb[i], ecp_matrix_Hb[j], dims=dims
#         )
#         ecp_difference_matrix_Hb[j, i] = ecp_difference_matrix_Hb[i, j]